In [1]:
import pandas as pd

tic='nasdaq'

# Location of dataset
#file = "C:\\Users\\KocurLocur\\Desktop\\jupyter\\TT_dax_data_new.txt"
file = "http://bpk.nazwa.pl/111/test.php?tic="+tic
TT_days=100

In [2]:
# Assign colum names to the dataset
names = ['Date', 'Price', 'Vol', 'T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11']
# Read dataset to pandas dataframe
df = pd.read_csv(file, names=names) 
#Check dayWeek
df['Date']=pd.to_datetime(df['Date'])
df['D']=df['Date'].dt.dayofweek
df.iloc[:,2:15]=df.iloc[:,2:15].astype(float)

In [3]:
#count percentage volume from day before
df['Vol']=df['Vol'].apply(pd.to_numeric, errors='coerce')

if (tic=='gold' or tic=='brent'):
    df['%_v1']=0.0
else:
    df['%_v1']=100*(1-df['Vol'].shift(-1)/df['Vol'])
    df['%_v1']=df['%_v1'].apply(pd.to_numeric, errors='coerce')

In [4]:
#shift values
for y in range(1,TT_days+1):
    for x in range (1,12):
        df['T'+str(x)+'_'+str(y)]=df['T'+str(x)].shift(-y) 

In [5]:
for y in range(1,6):
    df['%_p'+str(y)]=100*(1-df['Price'].shift(y)/df['Price'])
df.head(5)

,Date,Price,Vol,T1,T2,T3,T4,T5,T6,T7,...,T7_100,T8_100,T9_100,T10_100,T11_100,%_p1,%_p2,%_p3,%_p4,%_p5
0,2019-06-11,7513.85,162.19,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN
1,2019-06-10,7501.93,176.90,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,-0.158892,NaN,NaN,NaN,NaN
2,2019-06-07,7417.29,176.53,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,-1.141118,-1.301823,NaN,NaN,NaN
3,2019-06-06,7275.93,172.72,2.0,2.0,2.0,2.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,2.0,-1.942844,-3.106132,-3.269960,NaN,NaN
4,2019-06-05,7220.90,181.76,2.0,2.0,2.0,2.0,1.0,-2.0,-2.0,...,1.0,1.0,1.0,1.0,1.0,-0.762093,-2.719744,-3.891897,-4.056974,NaN


In [6]:
import numpy as np
df['Max']=df[["%_p1", "%_p2","%_p3","%_p4","%_p5"]].max(axis=1)
df['Min']=df[["%_p1", "%_p2","%_p3","%_p4","%_p5"]].min(axis=1)
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna()

In [7]:
df['y']=0
df.loc[(df['Min'] <=-1), 'y'] = 1
df.loc[(df['Max'] >=2 ), 'y'] = -1
df.loc[(df['Max'] >=2 ) & (df['Min'] <=-1 ), 'y'] = 2
df.head(5)

,Date,Price,Vol,T1,T2,T3,T4,T5,T6,T7,...,T10_100,T11_100,%_p1,%_p2,%_p3,%_p4,%_p5,Max,Min,y
5,2019-06-04,7166.75,217.16,2.0,2.0,2.0,2.0,1.0,-2.0,-2.0,...,1.0,1.0,-0.755573,-1.523424,-3.495866,-4.676876,-4.843199,-0.755573,-4.843199,1
6,2019-06-03,6978.02,245.97,2.0,2.0,2.0,2.0,-3.0,-3.0,-3.0,...,1.0,1.0,-2.704635,-3.480644,-4.269263,-6.295052,-7.508004,-2.704635,-7.508004,1
7,2019-05-31,7127.96,183.95,1.0,2.0,2.0,2.0,-2.0,-3.0,-3.0,...,-2.0,-2.0,2.103547,-0.544195,-1.303879,-2.075910,-4.059086,2.103547,-4.059086,2
8,2019-05-30,7245.40,135.81,1.0,2.0,2.0,2.0,-2.0,-2.0,-2.0,...,-2.0,-2.0,1.620890,3.690341,1.085516,0.338146,-0.421371,3.690341,-0.421371,-1
9,2019-05-29,7216.86,195.44,1.0,2.0,2.0,2.0,-2.0,-3.0,-3.0,...,-2.0,-2.0,-0.395463,1.231838,3.309473,0.694346,-0.055980,3.309473,-0.395463,-1


In [8]:
X = df.iloc[:, 3:(16+11*TT_days)]
y=df.iloc[:,-1]
X.head(5)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T2_100,T3_100,T4_100,T5_100,T6_100,T7_100,T8_100,T9_100,T10_100,T11_100
5,2.0,2.0,2.0,2.0,1.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0
6,2.0,2.0,2.0,2.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,...,1.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,2.0,2.0,2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,...,1.0,1.0,3.0,3.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
8,1.0,2.0,2.0,2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,...,1.0,1.0,3.0,3.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
9,1.0,2.0,2.0,2.0,-2.0,-3.0,-3.0,-3.0,-3.0,-3.0,...,1.0,1.0,2.0,2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0


In [9]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 
X_train , y_train
X_test.size
#y_test.value_counts()
#y_test.values.reshape(-1),y_test


644427

In [10]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test) 

In [11]:
from sklearn.neural_network import MLPClassifier 
mlp = MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=1000,verbose=True)  
mlp.fit(X_train, y_train.values.reshape(-1))  
#mlp.fit(X_train, y_train)


Iteration 1, loss = 1.25875388
Iteration 2, loss = 0.84646906
Iteration 3, loss = 0.63405359
Iteration 4, loss = 0.44086405
Iteration 5, loss = 0.29309261
Iteration 6, loss = 0.21156367
Iteration 7, loss = 0.16397363
Iteration 8, loss = 0.14014502
Iteration 9, loss = 0.13793509
Iteration 10, loss = 0.11126235
Iteration 11, loss = 0.08460587
Iteration 12, loss = 0.06473236
Iteration 13, loss = 0.08460161
Iteration 14, loss = 0.08724828
Iteration 15, loss = 0.09584682
Iteration 16, loss = 0.06450366
Iteration 17, loss = 0.05896267
Iteration 18, loss = 0.03316921
Iteration 19, loss = 0.02947576
Iteration 20, loss = 0.02956000
Iteration 21, loss = 0.02857982
Iteration 22, loss = 0.02833963
Iteration 23, loss = 0.02727407
Iteration 24, loss = 0.03206874
Iteration 25, loss = 0.01844860
Iteration 26, loss = 0.01680682
Iteration 27, loss = 0.00757290
Iteration 28, loss = 0.00527711
Iteration 29, loss = 0.01018479
Iteration 30, loss = 0.01918974
Iteration 31, loss = 0.02274926
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 500, 500), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [12]:
predictions = mlp.predict(X_test) 

In [302]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
a,b,c,d,=confusion_matrix(y_test,predictions)

[[ 33  16  10   5]
 [ 10 151  23   0]
 [  1  46 113   1]
 [  3   0   4   3]]
              precision    recall  f1-score   support

          -1       0.70      0.52      0.59        64
           0       0.71      0.82      0.76       184
           1       0.75      0.70      0.73       161
           2       0.33      0.30      0.32        10

   micro avg       0.72      0.72      0.72       419
   macro avg       0.62      0.58      0.60       419
weighted avg       0.72      0.72      0.71       419



In [303]:
prec=round((2*a[0]-b[0]*0.5-c[0]-d[0])/(a[0]+b[0]+c[0]+d[0]),4)

prec,(2*a[0]-b[0]*0.5-c[0]-d[0])/sum(a)


(1.2128, 0.890625)

In [304]:
import datetime
now = datetime.datetime.now()
filename='_'.join((tic,now.strftime('%Y%m%d'),str(a[0]),str(b[0]),str(c[0]),str(int(prec*1000))))

In [305]:
from joblib import dump,load
dump(mlp,filename+'_short_mlp.joblib')
dump(scaler,filename+'_short_scaler.joblib')

['dji_20190601_33_10_1_1212_short_scaler.joblib']

In [218]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#clf = tree.DecisionTreeClassifier(criterion='gini')
clf= RandomForestClassifier(n_estimators=1000, random_state=0,verbose=1) 

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train.values.reshape(-1))


#Predict the response for test dataset
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
a,b,c,d,=confusion_matrix(y_test,y_pred)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   20.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[[ 26  19  20   0]
 [  5 152  31   0]
 [  4  29 121   0]
 [  3   0   9   0]]
              precision    recall  f1-score   support

          -1       0.68      0.40      0.50        65
           0       0.76      0.81      0.78       188
           1       0.67      0.79      0.72       154
           2       0.00      0.00      0.00        12

   micro avg       0.71      0.71      0.71       419
   macro avg       0.53      0.50      0.50       419
weighted avg       0.69      0.71      0.70       419



[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.3s finished
c:\users\kocurlocur\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [219]:
prec=round((2*a[0]-b[0]*0.5-c[0]-d[0])/(a[0]+b[0]+c[0]+d[0]),4)

prec,(2*a[0]-b[0]*0.5-c[0]-d[0])/sum(a)

(1.1184, 0.6538461538461539)

In [220]:
dump(clf,tic+'_short_forest.joblib')

['dji_short_forest.joblib']

In [221]:
#import graphviz 
#dot_data = tree.export_graphviz(clf, out_file=None,class_names=['-1','0','1','2']) 
#graph = graphviz.Source(dot_data) 
#graph.render(tic) 

In [222]:
'''
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())
'''

'\nfrom sklearn.externals.six import StringIO  \nfrom IPython.display import Image  \nfrom sklearn.tree import export_graphviz\nimport pydotplus\ndot_data = StringIO()\nexport_graphviz(clf, out_file=dot_data,  \n                filled=True, rounded=True,\n                special_characters=True)\ngraph = pydotplus.graph_from_dot_data(dot_data.getvalue())  \nImage(graph.create_png())\n'